In [27]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws)

#Create R environment from dockerfile
from azureml.core import Environment
curated_env = Environment.get(ws, 'AzureML-minimal-ubuntu18.04-py37-cpu-inference')

env = curated_env.clone('r_train_env1')
env.docker.enabled = True


# if you are using base image from a Dockerfile
env.docker.base_image = None
env.docker.base_dockerfile = '/docker-context/Dockerfile'

# The user_managed_dependencies flag to True will use your custom image's built-in Python environment. 
env.python.user_managed_dependencies = True
env.register(ws)

Workspace.create(name='wu2modtimesmlsw', subscription_id='7fd76d0f-84f2-498b-a997-e0d059af5ce1', resource_group='wu2modtimerg')


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "FROM rocker/tidyverse:4.0.0-ubuntu18.04\n\n# Install python\nRUN apt-get update -qq && \\\n apt-get install -y python3-pip tcl tk libz-dev libpng-dev\n\nRUN ln -f /usr/bin/python3 /usr/bin/python\nRUN ln -f /usr/bin/pip3 /usr/bin/pip\nRUN pip install -U pip\n\n# Install azureml-mlflow\nRUN pip install azureml-mlflow\n\n# Create link for python\nRUN ln -f /usr/bin/python3 /usr/bin/python\n\n# Install additional R packages\nRUN R -e \"install.packages(c('mlflow'), repos = 'https://cloud.r-project.org/')\"\nRUN R -e \"install.packages(c('carrier'), repos = 'https://cloud.r-project.org/')\"\nRUN R -e \"install.packages(c('optparse'), repos = 'https://cloud.r-project.org/')\"\nRUN R -e \"install.packages(c('tcltk2'), repos = 'https://cloud.r-project.org/

In [28]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

num_nodes = 1

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_Dv2", max_nodes=num_nodes  
    )
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
